<a href="https://colab.research.google.com/github/ramkamal2000/cs6910_assignment3/blob/main/attn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Installing Required Packages

In [ ]:
!pip install wandb

# Importing Required Libraries

In [ ]:
import tarfile
import os
import pandas as pd
import keras
import numpy as np
import wandb
import tensorflow as tf
import pickle
import shutil
from keras import layers
from keras.layers import LSTM, Dense, Embedding, Input
from keras.models import Model
from keras.utils.vis_utils import plot_model
from tqdm.auto import tqdm
from keras.layers import Lambda
from keras import backend as K
import datetime
from math import ceil
from pprint import pprint
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Concatenate, TimeDistributed,Dropout
from tensorflow.python.keras.models import Model

# Setting Current Directory

In [ ]:
# MAKE REQUIRED CHANGES

# dir = '/content'
dir = '/kaggle/working'
# dir = os.getcwd()

# Downloading Dataset

In [ ]:
!wget -nc https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

if not os.path.isdir(dir + '/dakshina_dataset_v1.0'):
  tarfile.open(dir + "/dakshina_dataset_v1.0.tar").extractall()

# Logging Onto wandb

In [ ]:
wandb.login(key='14394907543f59ea21931529e34b4d80d2ca8c9c')
wandb.init(project='final_attn')

# Loading Data

In [ ]:
class data_loader():

  @staticmethod
  def _load_raw_df(languages = ["ta"]):
    lex = dict()
    lex['train'], lex['val'], lex['test'] = [], [], [] 
    column_names = ['output', 'input', 'count']
    
    for la in languages:
      lex['train'].append(pd.read_csv(dir + '/dakshina_dataset_v1.0/'+la+'/lexicons/'+la+'.translit.sampled.train.tsv', sep='\t', header=None, names=column_names))
      lex['val'].append(pd.read_csv(dir + '/dakshina_dataset_v1.0/'+la+'/lexicons/'+la+'.translit.sampled.dev.tsv', sep='\t', header=None, names=column_names))
      lex['test'].append(pd.read_csv(dir + '/dakshina_dataset_v1.0/'+la+'/lexicons/'+la+'.translit.sampled.test.tsv', sep='\t', header=None, names=column_names))

    lex['train'] = pd.concat(lex['train'])
    lex['val'] = pd.concat(lex['val'])
    lex['test'] = pd.concat(lex['test'])

    return lex    

  @staticmethod
  def _make_final_df(lex):
    
    for div in ['train', 'val']:
    
      # removing non max transliterations
      idx = lex[div].groupby(['input'])['count'].transform(max) == lex[div]['count']
      lex[div] = lex[div][idx].reset_index(drop=True)

      # calclulating difference in lengths of various transliterations
      lex[div]['input_len'] = lex[div].apply(lambda x: len(str(x['input'])), axis=1)
      lex[div]['output_len'] = lex[div].apply(lambda y: len(str(y['output'])), axis=1)
      lex[div]['mod_dif'] = lex[div].apply(lambda z: abs(z['input_len'] - z['output_len']), axis=1) 

      # removing transliterations that vary by a lot in length
      idx = lex[div].groupby(['input'])['mod_dif'].transform(min) == lex[div]['mod_dif']
      lex[div] = lex[div][idx].reset_index(drop=True)

      # removing duplicates if any remain
      lex[div].drop_duplicates(subset='input', keep='first', inplace=True)

      # removing redundant columns
      lex[div].drop(labels=['count', 'input_len', 'output_len', 'mod_dif'], inplace=True, axis=1)

      # shuffling the dataset i.e. rows of the dataset
      lex[div] = lex[div].sample(frac=1, random_state=6910)
      lex[div] = lex[div].reset_index(drop=True)

    lex['test'] = lex['test'].sample(frac=1, random_state=6910)
    lex['test'].drop(labels=['count'], axis=1, inplace=True)
    lex['test'] = lex['test'].reset_index(drop=True)
    return lex

  @staticmethod
  def _generate_batch(X, y, data_dict, num_decoder_tokens, batch_size = 1):

    while True:
        for j in range(0, len(X), batch_size):
            
            # placeholder data structures
            encoder_input_data = np.zeros((batch_size, data_dict['max_source_length']),dtype='float32')
            decoder_input_data = np.zeros((batch_size, data_dict['max_target_length']),dtype='float32')
            decoder_target_data = np.zeros((batch_size, data_dict['max_target_length'], num_decoder_tokens),dtype='float32')

            # assessing one batch at a time
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):

                for t, word in enumerate(input_text):
                  encoder_input_data[i, t] = word
                for t, word in enumerate(target_text):
                    if t<len(target_text)-1:
                        # decoder input sequence
                        # does not include the <EOW> token
                        decoder_input_data[i, t] = word 
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the <SOW> token
                        decoder_target_data[i, t - 1, word] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

  @staticmethod
  def _generate_batch_greedy(X, y, data_dict, num_decoder_tokens, batch_size = 1):

    while True:
        for j in range(0, len(X), batch_size):

            # placeholder data structures
            encoder_input_data = np.zeros((batch_size, data_dict['max_source_length']),dtype='float32')
            decoder_input_data = np.zeros((batch_size, 1),dtype='float32')
            decoder_target_data = np.zeros((batch_size, data_dict['max_target_length'], num_decoder_tokens),dtype='float32')
            
            # assessing one batch at a time
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text):
                  encoder_input_data[i, t] = word
                for t, word in enumerate(target_text):
                    if t==0 :
                        decoder_input_data[i, t] = 1 # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        decoder_target_data[i, t - 1, word] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
# tokenizer class
class Tokenizer:

  def __init__(self, df):

    self.start_token = '<SOW>'
    self.stop_token = '<EOW>'
    self.unknown_token = '<UNK>'

    self.input_corpus = [self.start_token, self.stop_token, self.unknown_token]
    self.output_corpus = [self.start_token, self.stop_token, self.unknown_token]

    input_words = df.input.tolist()
    output_words = df.output.tolist()

    for word in input_words:
      tokens = str(word)
      for token in tokens:
        if token not in self.input_corpus:
          self.input_corpus.append(token)

    for word in output_words:
      tokens = str(word)
      for token in tokens:
        if token not in self.output_corpus:
          self.output_corpus.append(token)
    
    self.encode_dict_input = {self.input_corpus[i] : i+1 for i in range(len(self.input_corpus))}
    self.decode_dict_input = {k:v for v,k in self.encode_dict_input.items()}
    
    self.encode_dict_output = {self.output_corpus[i] : i+1 for i in range(len(self.output_corpus))}
    self.decode_dict_output = {k:v for v,k in self.encode_dict_output.items()}
    self.decode_dict_output.update({2:''})

  # takes in lists of words and returns lists of integers
  def encode(self, X, mode='input'):

    if (mode=='input'):
      input_list = []
      for word in X:
        word = str(word)
        integer_list =np.array([self.encode_dict_input.get(token, self.encode_dict_input[self.unknown_token]) for token in word])
        input_list.append(integer_list)
      
      return input_list
    
    if (mode=='output'):
      output_list = []
      for word in X:
        word = str(word)
        integer_list = np.array([self.encode_dict_output[self.start_token]] + [self.encode_dict_output.get(token, self.encode_dict_output[self.unknown_token]) for token in word] + [self.encode_dict_output[self.stop_token]])
        output_list.append(integer_list)
      
      return output_list
    
  # takes in lists of integers and returns lists of words
  def decode(self, X, mode='input'):

    if (mode=='input'):
      input_list = []
      for integers in X:
        token_list = [self.decode_dict_input.get(integer, '') for integer in integers] 
        input_list.append(''.join(token_list))
      
      return input_list

    if (mode=='output'):
      output_list = []
      for integers in X:
        token_list = [self.decode_dict_output.get(integer, '') for integer in integers[:-1]] 
        output_list.append(''.join(token_list))
      
      return output_list

In [ ]:
# final function that returns the required nested datastructure
def return_data_dict(languages=["ta"], batch_size=32):

  lex = data_loader._load_raw_df(languages)
  lex = data_loader._make_final_df(lex)

  data_dict = dict()

  df_train = lex['train']
  df_val = lex['val']
  df_test = lex['test']

  tk = Tokenizer(df_train)

  data_dict['in_size'] = len(tk.input_corpus) + 1
  data_dict['out_size'] = len(tk.output_corpus) + 1

  X_train = tk.encode(df_train.input.tolist(), mode='input')
  Y_train = tk.encode(df_train.output.tolist(), mode='output')
  
  X_val = tk.encode(df_val.input.tolist(), mode='input')
  Y_val = tk.encode(df_val.output.tolist(), mode='output')
  
  X_test = tk.encode(df_test.input.tolist(), mode='input')
  Y_test = tk.encode(df_test.output.tolist(), mode='output')


  data_dict['train'], data_dict['val'], data_dict['test']= dict(), dict(), dict()


  data_dict['train']['df'] = df_train
  data_dict['val']['df'] = df_val
  data_dict['test']['df'] = df_test


  data_dict['train']['max_source_length'] = np.max(np.array([len(x) for x in X_train]))
  data_dict['train']['max_target_length'] = np.max(np.array([len(x) for x in Y_train]))
  
  data_dict['val']['max_source_length'] = np.max(np.array([len(x) for x in X_val]))
  data_dict['val']['max_target_length'] = np.max(np.array([len(x) for x in Y_test]))
  
  data_dict['test']['max_source_length'] = np.max(np.array([len(x) for x in X_test]))
  data_dict['test']['max_target_length'] = np.max(np.array([len(x) for x in Y_test]))


  data_dict['max_source_length'] = max(data_dict['train']['max_source_length'], data_dict['val']['max_source_length'], data_dict['test']['max_source_length'])
  data_dict['max_target_length'] = max(data_dict['train']['max_target_length'], data_dict['val']['max_target_length'], data_dict['test']['max_target_length'])


  data_dict['train']['batch'] = data_loader._generate_batch(X_train, Y_train, data_dict, data_dict['out_size'], batch_size)
  data_dict['train']['batch_greedy'] = data_loader._generate_batch_greedy(X_train, Y_train, data_dict, data_dict['out_size'], batch_size)
  
  data_dict['val']['batch'] = data_loader._generate_batch(X_val, Y_val, data_dict, data_dict['out_size'], batch_size)
  data_dict['val']['batch_greedy'] = data_loader._generate_batch_greedy(X_val, Y_val, data_dict, data_dict['out_size'], batch_size)

  data_dict['test']['batch'] = data_loader._generate_batch(X_test, Y_test, data_dict, data_dict['out_size'], batch_size)  
  data_dict['test']['batch_greedy'] = data_loader._generate_batch_greedy(X_test, Y_test, data_dict, data_dict['out_size'], batch_size)    
  data_dict['test']['batch_1'] = data_loader._generate_batch_greedy(X_test, Y_test, data_dict, data_dict['out_size'], 1)


  data_dict['tokenizer'] = tk

  return data_dict

In [ ]:
# final data dictionary
data_dict = return_data_dict()

In [ ]:
# saving test dataframe for later use
# data_dict['test']['df'].to_csv('/content/drive/MyDrive/Colab Notebooks/DL3/test.csv')

# Attention - Question 5

In [ ]:
# attention layer
# obtained from https://github.com/thushv89/attention_keras/
class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
    """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.
        # print(input_shape[0][2],input_shape[1][2])
        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        # print("attn called")
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

In [ ]:
# single class for RNN
class attn_rnn():
    
  # initializing RNN
  def __init__(self,params) :
    in_size = params['data_dict']['in_size']
    out_size = params['data_dict']['out_size']
    dropout = params['dropout']
    embed_size = params['embed_size']
    rep_size = params['rep_size']

    encoder_inputs = Input(shape=(None,))
    decoder_inputs = Input(shape=(None,))


    encoder_emb = Embedding(in_size, embed_size, input_length=None,mask_zero=True)
    temp1 = encoder_emb(encoder_inputs)
    encoder_lstm = LSTM(rep_size, return_sequences=True, return_state=True,dropout=dropout)
    encoder_out, *encoder_state = encoder_lstm(temp1)
    

    decoder_emb = Embedding(out_size, embed_size , input_length=None,mask_zero=True)
    temp2 = decoder_emb(decoder_inputs)
    decoder_lstm = LSTM(rep_size, return_sequences=True, return_state=True,dropout=dropout)
    decoder_out, *decoder_state = decoder_lstm(temp2, initial_state=encoder_state)

    # Attention layer
    attn_layer = AttentionLayer()
    attn_out, attn_states = attn_layer([encoder_out, decoder_out])

    # Concat attention input and decoder GRU output
    decoder_concat_input = Concatenate(axis=-1)([decoder_out, attn_out])

    # Dense layer
    drop = Dropout(dropout)
    decoder_concat_input = drop(decoder_concat_input)
    dense = Dense(out_size, activation='softmax')
    dense_time = TimeDistributed(dense)
    decoder_pred = dense_time(decoder_concat_input)

    # Full model
    full_model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_pred)

    batch_size = 1

    """ Encoder (Inference) model """
    encoder_inf_inputs = Input(shape=(None,))
    temp3 = encoder_emb(encoder_inf_inputs)
    encoder_inf_out, *encoder_inf_state = encoder_lstm(temp3)
    encoder_model = Model(inputs=encoder_inf_inputs, outputs=[encoder_inf_out, encoder_inf_state])

    """ Decoder (Inference) model """
    decoder_inf_inputs = Input(shape=(None,))
    encoder_inf_states = Input(shape=(None, rep_size))
    decoder_init_stateh = Input(shape=(rep_size,))
    decoder_init_statec = Input(shape=(rep_size,))
    decoder_init_state = [decoder_init_stateh,decoder_init_statec]
    temp = decoder_emb(decoder_inf_inputs)
    decoder_inf_out, *decoder_inf_state = decoder_lstm(temp, initial_state=decoder_init_state)
    attn_inf_out, attn_inf_states = attn_layer([encoder_inf_states, decoder_inf_out])
    decoder_inf_concat = Concatenate(axis=-1)([decoder_inf_out, attn_inf_out])
    decoder_inf_pred = TimeDistributed(dense)(decoder_inf_concat)
    decoder_model = Model(inputs=[encoder_inf_states, decoder_init_state, decoder_inf_inputs],
                          outputs=[decoder_inf_pred, attn_inf_states, decoder_inf_state])

    self.model = full_model
    self.encoder = encoder_model
    self.decoder = decoder_model

  # compile and fit the model
  def compile_and_fit(self,data_dict,params):

    self.model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

    self.model.summary()
    batch_size=params['batch_size']
    epochs=params['num_epochs']
    val_samples = len(data_dict['val']['df'])
    train_samples = len(data_dict['train']['df'])

    self.model.fit_generator(generator = data_dict['train']['batch'],
                 steps_per_epoch=train_samples//batch_size,
                 epochs=epochs,
                 validation_data=data_dict['val']['batch'],
                 validation_steps=val_samples//batch_size
                 )
    
  # evaluate the test dataset
  def evaluate(self,data_dict,filename) :
    generator = data_dict['test']['batch_1']
    tk = data_dict['tokenizer']
    acc = 0
    num = len(data_dict['test']['df'])
    
    X = []
    Y_true = []
    Y_pred = []
    attn = []
    for j in tqdm(range(num)) :
      (a,b),c = next(generator)
      g = [np.argmax(c,axis=-1)[0]]
      enc_outs, enc_last_state = self.encoder.predict(a)
      dec_state = enc_last_state
      attention_weights = []
      word = []
      for i in range(23):

          dec_out, attention, dec_state = self.decoder.predict([enc_outs, dec_state, b])
          dec_ind = np.argmax(dec_out, axis=-1)

          word.append(dec_ind[0][0])
          b = dec_ind
          attention_weights.append(attention)
          if dec_ind[0][0] == 2 :
            break
      str1 = tk.decode(g,mode='output')
      str2 = tk.decode([word],mode='output')
      str3 = tk.decode(a)
      
      ###############################################################
      X.append(str3[0])
      Y_true.append(str1[0])
      Y_pred.append(str2[0])

      attn.append(attention_weights)
      # print(word)
      # print(str1[0],str2[0])
      if str1[0] == str2[0] :
        acc += 1
   
    df = pd.DataFrame({
      'X': X,
      'Y_true': Y_true,
      'Y_pred': Y_pred
    })

    df.to_csv(filename)
    try:
      wandb.log({'attn_greedy_csv': wandb.Table(dataframe=df)})
    except:
      pass

    with open('/kaggle/working/attm_weights', 'wb') as f:
        pickle.dump(attn, f)
    
    
    print("Test accuracy : ",acc/num)

  # helper function to retrieve weights
  def get_attn_weights(self, a, b) :
    enc_outs, enc_last_state = self.encoder.predict(a)
    dec_state = enc_last_state
    attention_weights = []
    word = []
    for i in range(23):

        dec_out, attention, dec_state = self.decoder.predict([enc_outs, dec_state, b])
        dec_ind = np.argmax(dec_out, axis=-1)

        word.append(dec_ind[0][0])
        b = dec_ind
        attention_weights.append((dec_ind, attention))
        if dec_ind[0][0] == 2 :
          break

    return attention_weights


# Attention Model Params

In [ ]:
params = {
    'rep_size' : 128,
    'embed_size' : 16,
    'dropout' : 0.5,
    'num_epochs' : 15,
    'data_dict' : data_dict,
    'batch_size' : 32
}

In [ ]:
# initializing and fitting model
network = attn_rnn(params)
network.compile_and_fit(data_dict,params)

In [ ]:
# evaluate test data and publish output csv
network.evaluate(data_dict,dir+'/attn_greedy.csv')

In [ ]:
# beam search function
def decode_sequence_beam_attn(input_seq,target_seq, k, encoder_model, decoder_model, tk, max_target_length=20, getall=False,alpha=0.7):
    # encode the input as state vectors
    enc_outs,states_value = encoder_model.predict(input_seq)
    # generate empty target sequence of length 1.
    # populate the first character of target sequence with the start character.
    run_condition = [True for i in range(k)]
    # print(len(states_value))
    # print([target_seq] + [states_value])
    results, attn,states_values_temp = decoder_model.predict([enc_outs , states_value,target_seq])
    output_tokens = results
    # print(results)
    states_values_k = [states_values_temp for i in range(k)]
    #get topk indices
    ind = np.argpartition(np.array(output_tokens[0, -1, :]), -k)[-k:]
    # print(ind)
    bestk_ind = ind
    output_tokens = np.array(output_tokens[0, -1, :])
    bestk_prob = np.log(output_tokens[ind])
    bestk_tot = [[bestk_ind[i]] for i in range(k)]
    # print(bestk_tot)
    # print(bestk_prob)

    
    while any(run_condition):
        # print("##############")
        bestk_tot_new = []
        bestk_prob_new = []
        states_values_k_new = []
        for i in range(k) :
            # print("**")
            if run_condition[i] :
                a = bestk_tot[i]
                b = bestk_prob[i]
                target_seq[0,0] = a[-1]
                results,attn,states_values_temp = decoder_model.predict([enc_outs,states_values_k[i],target_seq],batch_size=1)

                output_tokens = results

                states_values_k_temp = [states_values_temp for m in range(k)]

                states_values_k_new += states_values_k_temp
                ind = np.argpartition(np.array(output_tokens[0, -1, :]), -k)[-k:]
                bestk_ind = ind
                output_tokens = np.array(output_tokens[0, -1, :])
                bestk_prob_temp = output_tokens[ind]
                # print(np.log(bestk_prob_temp))
                bestk_tot_temp = [a+[bestk_ind[j]] for j in range(k)]
                # print(bestk_tot_temp)
                bestk_prob_temp2 = [(b*(np.power(len(bestk_tot_temp[j])-1,alpha)) + np.log(bestk_prob_temp[j]))/(np.power(len(bestk_tot_temp[j]),alpha)) for j in range(k)]
                # print(bestk_prob_temp2)
                bestk_prob_new += bestk_prob_temp2
                bestk_tot_new += bestk_tot_temp
            
            else :
                a = bestk_tot[i]
                b = bestk_prob[i]
                bestk_tot_new += [bestk_tot[i]]
                bestk_prob_new += [b]
                states_values_k_new += [states_values_k[i]]

        bestk_prob_new = np.array(bestk_prob_new)
        # print(len(bestk_prob_new),len(bestk_tot_new),len(states_values_k_new))
        ind = np.argpartition(bestk_prob_new,-k)[-k:]
        bestk_tot = [bestk_tot_new[i] for i in ind]
        states_values_k = [states_values_k_new[i] for i in ind]
        bestk_prob = bestk_prob_new[ind]
        run_condition = []
        for i in range(k) :
            a = bestk_tot[i]
            b = bestk_prob[i]
            if a[-1]!= 2 and len(a)<=max_target_length :
              run_condition.append(True)
            else :
              run_condition.append(False)

        # print(bestk_tot)

    final_words = []
    best_word = []
    best = -5.0
    for i in range(k) :
      a = bestk_tot[i]
      b = bestk_prob[i]
      final_words += [a]
      if b > best :
        best_word = [a]
        best = b

    if getall :
      return (tk.decode(final_words,'output'),best_word)
    else :
      return final_words,bestk_prob, best_word

In [ ]:
##run beam search

generator = data_dict['test']['batch_1']
tk = data_dict['tokenizer']
m2 = network.encoder
m3 = network.decoder
acc = 0
num = len(data_dict['test']['df'])
acc_k = 0
# num = 50
X = []
Y_true = []
Y_pred = []
for j in tqdm(range(num)) :
  (a,b),c = next(generator)
  g = [np.argmax(c,axis=-1)[0]]
  attention_weights = []
  K,word = decode_sequence_beam_attn(a,b,10,m2,m3,data_dict['tokenizer'],max_target_length=data_dict['max_target_length'],getall=True )
  str1 = tk.decode(g,mode='output')
  str2 = tk.decode(word,mode='output')
  str3 = tk.decode(a)
  X.append(str3[0])
  Y_true.append(str1[0])
  Y_pred.append(str2[0])

  if str1[0] in K :
    acc_k += 1
  
  if str1[0] == str2[0] :
    acc += 1
    
  if (j+1) %500 == 0 :
    print(acc/num)
    print(acc_k/num)

  df = pd.DataFrame({
    'X': X,
    'Y_true': Y_true,
    'Y_pred': Y_pred
  })

  df.to_csv(dir+'/attn_beam')
  try:
    wandb.log({'attn_beam_csv': wandb.Table(dataframe=df)})
  except:
    pass

print(acc/num)
print(acc_k/num)